In [1]:
!nvidia-smi

Mon Apr 18 04:59:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -U PyYAML
!pip install wandb

     |████████████████████████████████| 596 kB 4.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.8 MB 4.2 MB/s 
     |████████████████████████████████| 144 kB 17.9 MB/s 
     |████████████████████████████████| 181 kB 26.2 MB/s 
     |████████████████████████████████| 63 kB 878 kB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=f66ccc9d6aa2757e795933d74b39a70c3b189d2c11b35417dbc376942054bcbc
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


# Mount GDrive

In [ ]:
#@title Mount your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
%matplotlib inline
%load_ext autoreload
%autoreload 2
''
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/MILA/IFT6759/Project/src" #@param {type:"string"}
!ln -Ts "$folder" /content/src 2> /dev/null

# Add the assignment folder to Python path
if '/content/src' not in sys.path:
  sys.path.insert(0, '/content/src')

# Install requirements
#!pip install -qr /content/assignment/requirements.txt

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
  warnings.warn('CUDA is not available.')

# Load from GitHub



```
!git clone https://{TOKEN}@github.com/TimkLee/IFT6759.git
```

Create your own PAT over [here](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token)

In [3]:
#Enter your PAT here
TOKEN = ''

In [4]:
!git clone https://{TOKEN}@github.com/TimkLee/IFT6759.git

Cloning into 'IFT6759'...
remote: Enumerating objects: 589, done.
remote: Counting objects: 100% (589/589), done.
remote: Compressing objects: 100% (305/305), done.
remote: Total 589 (delta 411), reused 414 (delta 261), pack-reused 0
Receiving objects: 100% (589/589), 200.84 KiB | 2.61 MiB/s, done.
Resolving deltas: 100% (411/411), done.


In [5]:
%cd /content/IFT6759/src

/content/IFT6759/src


In [6]:
import argparse
import logging
import os
import torch
import yaml
import numpy as np
import pickle
import logging
import torch.nn.functional as F
import wandb

In [7]:
config_name = "Sup_Config0t2"

with open(f"./Config/{config_name}.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num = config['num'] #New addition just add the config Number in the YAML file, so that the WANDB run can be saved with the config number #Ex: num: 00 for config0
task = config["task"]
data_file = config["data"]
model_file = config["model"]
augment_file = config["augment"]
augment_strength = config["aug_strength"]
eval_file = config["eval"]
batch_size = config["batch_size"]
learn_rate = config["learning_rate"]
epoch = config["epoch"]
optimizer = config["optimizer"]
momentum = config["momentum"]
weight_decay = config["weight_decay"]
seed = config["seed"]
epoch = config["epoch"]

In [9]:
config

{'aug_strength': None,
 'augment': None,
 'batch_size': 120,
 'data': 'cifar10',
 'epoch': 10,
 'eval': 'Acc',
 'learning_rate': 0.1,
 'model': 'resnet20',
 'momentum': 0.9,
 'num': 0,
 'optimizer': 'sgd',
 'seed': 6759,
 'task': 'super',
 'weight_decay': 0.0005}

In [10]:
# logging.info(f"==========Dataset: {data_file}==========")
print(f"==========Dataset: {data_file}==========")
data_file_path = f"Data.{data_file}"
_temp = __import__(name=data_file_path, fromlist=['Data_Load'])
Data_Load = _temp.Data_Load

==========Dataset: cifar10==========


In [11]:
if augment_file == None:
    print("No augmentation method selected")
else:
    Aug = []
    for i in range(len(augment_file)):
        # logging.info(f"==========Augmentation Methods: {augment_file[i]}, with a strength value of {augment_strength[i]}==========")
        print(f"==========Augmentation Methods: {augment_file[i]}, with a strength value of {augment_strength[i]}==========")
        augment_file_path = f"Augmentation.{augment_file[i]}"
        _temp = __import__(name=augment_file_path, fromlist=['Aug'])
        Aug.append(_temp.Aug)

No augmentation method selected


In [12]:
# Importing the model class
# logging.info(f"==========Model Selected: {model_file}==========")
print(f"==========Model Selected: {model_file}==========")
model_file_path = f"Model.{model_file}"
_temp = __import__(name=model_file_path, fromlist=['ModelClass'])
ModelClass = _temp.ModelClass

==========Model Selected: resnet20==========


In [13]:
# # Importing the evaluation methods
# logging.info(f"==========Evaluation Method: {eval_file}==========")
# eval_file_path = f"Evaluation.{eval_file}"
# _temp = __import__(name=eval_file_path, fromlist=['Eval'])
# Eval = _temp.Eval  

In [14]:
labelledloader, unlabelledloader, validloader, testloader = Data_Load(task = task, batch_size = batch_size, seed = seed)
logging.info("Dataloader ready")    

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./Data/Cifar10_Data/cifar-10-python.tar.gz to ./Data/Cifar10_Data
Files already downloaded and verified


INFO:root:Data Extracted
INFO:root:Data split for supervised learning task
INFO:root:Data Loaded
INFO:root:Dataloader ready


##Super

In [ ]:
#NON WANDB RUNS
if (task == "super"):  
    torch.manual_seed(seed)
    np.random.seed(seed)

    train_tot_accs, valid_tot_accs = [], []
    train_tot_losses, valid_tot_losses = [], []

    Model = ModelClass(device=device, num_classes=len(testloader.dataset.classes), optimizer=optimizer,lr=learn_rate,weight_decay=weight_decay,momentum=momentum)
    Model = Model.to(device=device)
    example_ct = 0
    batch_log_interval=25
    
    #with wandb.init(project=f"Supervised Learning {model_file} {data_file}", entity='ift6759-aiadlp', job_type="train", name=f'Config_{num}') as run:
    for ep in range(epoch):
        Model.train()
        #logging.info(f"==========Supervised Learning Epoch Number: {ep+1}/{epoch}==========")
        print(f"==========Supervised Learning Epoch Number: {ep+1}/{epoch}==========")
        train_accs, valid_accs = [], []
        train_losses, valid_losses = [], []
        
        if ep == epoch/2:
            learn_rate = learn_rate/10
            print(f"Update Learning Rate: {learn_rate}")
            Model.update_lr(learn_rate)

        if ep == 3*epoch/4:
            learn_rate = learn_rate/10
            print(f"Update Learning Rate: {learn_rate}")
            Model.update_lr(learn_rate)

        for idx, batch in enumerate(labelledloader):
            data, target = batch
            data = data.to(device=device)
            labels = F.one_hot(target, num_classes = 10).float().to(device=device)

            batch_len = data.shape[0]
            aug_num = []
            
            if augment_file != None:

                for i in range(len(augment_strength)):
                    aug_num.append(augment_strength[i]*batch_len)
                
                if len(aug_num) != 1:
                    aug_num = torch.tensor(aug_num)
                    aug_ind = torch.cumsum(aug_num,0).int()
                else:
                    aug_ind = aug_num

                for i in range(len(Aug)):
                    if i == 0:
                        temp_Aug, temp_label = Aug[i](data[0:aug_ind[i]],labels[0:aug_ind[i]])
                        Aug_data = temp_Aug
                        Aug_labels = temp_label
                    else:
                        temp_Aug, temp_label = Aug[i](data[aug_ind[i-1]:aug_ind[i]],labels[aug_ind[i-1]:aug_ind[i]])
                        Aug_data = torch.cat((Aug_data, temp_Aug), 0)
                        Aug_labels = torch.cat((Aug_labels, temp_label), 0)

            else:
                # Aug_data = torch.cat((data,data,data,data),0)
                # Aug_labels = torch.cat((labels,labels,labels,labels),0)
                # Aug_data = data
                # Aug_labels = labels
                Aug_data = torch.cat((data,data),0)
                Aug_labels = torch.cat((labels,labels),0)

            #print(Aug_data.shape)

            loss = Model.train_sup_up(Aug_data,Aug_labels)
            #print(loss)
            example_ct += len(Aug_data)

            if idx % batch_log_interval == 0:
                #Logging into WANDB
                print('Train Epoch: {} \tLoss: {:.6f}'.format(ep+1, loss))

                loss = float(loss)
                #wandb.log({"epoch": ep+1, "train/loss": loss}, step=example_ct)
                print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")
                
        #Logging into WANDB
        train_loss, train_accuracy = Model.test(labelledloader)
        valid_loss, valid_accuracy = Model.test(validloader)
        train_loss, train_accuracy, valid_loss, valid_accuracy = float(train_loss), float(train_accuracy), float(valid_loss), float(valid_accuracy)
        #wandb.log({"epoch": ep+1, "train/loss": train_loss, "train/accuracy": train_accuracy, "validation/loss": valid_loss, "validation/accuracy": valid_accuracy}, step=example_ct)	
        print(f"Train Loss/accuracy after " + str(example_ct).zfill(5) + f" examples: {train_loss:.3f}/{train_accuracy:.3f}")
        print(f"Validation Loss/accuracy after " + str(example_ct).zfill(5) + f" examples: {valid_loss:.3f}/{valid_accuracy:.3f}")    

    #Evaluation:
    #Model.evaluation(testloader, project=f"Supervised Learning {model_file} {data_file}", entity='ift6759-aiadlp', name=f'Config_{num}')

In [16]:
if (task == "super"):  
    torch.manual_seed(seed)
    np.random.seed(seed)

    train_tot_accs, valid_tot_accs = [], []
    train_tot_losses, valid_tot_losses = [], []

    Model = ModelClass(device=device, num_classes=len(testloader.dataset.classes), optimizer=optimizer,lr=learn_rate,weight_decay=weight_decay,momentum=momentum)
    Model = Model.to(device=device)
    example_ct = 0
    batch_log_interval=25
    
    with wandb.init(project=f"Supervised Learning {model_file} {data_file}", entity='ift6759-aiadlp', job_type="train", name=f'Config_{num}') as run:
        for ep in range(epoch):
            Model.train()
            #logging.info(f"==========Supervised Learning Epoch Number: {ep+1}/{epoch}==========")
            print(f"==========Supervised Learning Epoch Number: {ep+1}/{epoch}==========")
            train_accs, valid_accs = [], []
            train_losses, valid_losses = [], []
            
            if ep == epoch/2:
                learn_rate = learn_rate/10
                print(f"Update Learning Rate: {learn_rate}")
                Model.update_lr(learn_rate)

            if ep == 3*epoch/4:
                learn_rate = learn_rate/10
                print(f"Update Learning Rate: {learn_rate}")
                Model.update_lr(learn_rate)

            for idx, batch in enumerate(labelledloader):
                data, target = batch
                data = data.to(device=device)
                labels = F.one_hot(target, num_classes = 10).float().to(device=device)

                batch_len = data.shape[0]
                aug_num = []
                
                if augment_file != None:

                    for i in range(len(augment_strength)):
                        aug_num.append(augment_strength[i]*batch_len)
                    
                    if len(aug_num) != 1:
                        aug_num = torch.tensor(aug_num)
                        aug_ind = torch.cumsum(aug_num,0).int()
                    else:
                        aug_ind = aug_num

                    for i in range(len(Aug)):
                        if i == 0:
                            temp_Aug, temp_label = Aug[i](data[0:aug_ind[i]],labels[0:aug_ind[i]])
                            Aug_data = temp_Aug
                            Aug_labels = temp_label
                        else:
                            temp_Aug, temp_label = Aug[i](data[aug_ind[i-1]:aug_ind[i]],labels[aug_ind[i-1]:aug_ind[i]])
                            Aug_data = torch.cat((Aug_data, temp_Aug), 0)
                            Aug_labels = torch.cat((Aug_labels, temp_label), 0)

                else:
                    # Aug_data = torch.cat((data,data,data,data),0)
                    # Aug_labels = torch.cat((labels,labels,labels,labels),0)
                    # Aug_data = data
                    # Aug_labels = labels
                    Aug_data = torch.cat((data,data),0)
                    Aug_labels = torch.cat((labels,labels),0)

                #print(Aug_data.shape)

                loss = Model.train_sup_up(Aug_data,Aug_labels)
                #print(loss)
                example_ct += len(Aug_data)

                if idx % batch_log_interval == 0:
                    #Logging into WANDB
                    print('Train Epoch: {} \tLoss: {:.6f}'.format(ep+1, loss))

                    loss = float(loss)
                    wandb.log({"epoch": ep+1, "train/loss": loss}, step=example_ct)
                    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")
                    
            #Logging into WANDB
            train_loss, train_accuracy = Model.test(labelledloader)
            valid_loss, valid_accuracy = Model.test(validloader)
            train_loss, train_accuracy, valid_loss, valid_accuracy = float(train_loss), float(train_accuracy), float(valid_loss), float(valid_accuracy)
            wandb.log({"epoch": ep+1, "train/loss": train_loss, "train/accuracy": train_accuracy, "validation/loss": valid_loss, "validation/accuracy": valid_accuracy}, step=example_ct)	
            print(f"Train Loss/accuracy after " + str(example_ct).zfill(5) + f" examples: {train_loss:.3f}/{train_accuracy:.3f}")
            print(f"Validation Loss/accuracy after " + str(example_ct).zfill(5) + f" examples: {valid_loss:.3f}/{valid_accuracy:.3f}")    

    #Evaluation:
    Model.evaluation(testloader, project=f"Supervised Learning {model_file} {data_file}", entity='ift6759-aiadlp', name=f'Config_{num}')

==========Supervised Learning Epoch Number: 1/10==========
Train Epoch: 1 	Loss: 2.623647
Loss after 00240 examples: 2.624
Train Epoch: 1 	Loss: 1.993307
Loss after 06240 examples: 1.993
Train Epoch: 1 	Loss: 1.770578
Loss after 12240 examples: 1.771
Train Epoch: 1 	Loss: 1.583547
Loss after 18240 examples: 1.584
Train Epoch: 1 	Loss: 1.589854
Loss after 24240 examples: 1.590
Train Epoch: 1 	Loss: 1.483825
Loss after 30240 examples: 1.484
Train Epoch: 1 	Loss: 1.394876
Loss after 36240 examples: 1.395
Train Epoch: 1 	Loss: 1.446796
Loss after 42240 examples: 1.447
Train Epoch: 1 	Loss: 1.433171
Loss after 48240 examples: 1.433
Train Epoch: 1 	Loss: 1.467457
Loss after 54240 examples: 1.467
Train Epoch: 1 	Loss: 1.279794
Loss after 60240 examples: 1.280
Train Epoch: 1 	Loss: 1.337917
Loss after 66240 examples: 1.338
Train Epoch: 1 	Loss: 1.373053
Loss after 72240 examples: 1.373
Train Epoch: 1 	Loss: 1.330713
Loss after 78240 examples: 1.331
Train Epoch: 1 	Loss: 1.225352
Loss after 842

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train/accuracy,▁▃▄▆▆█████
train/loss,█▆▆▆▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▁▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▁
validation/accuracy,▁▄▅▇▇█████
validation/loss,█▅▄▂▂▁▁▁▁▁
epoch,10
train/accuracy,91.86222
train/loss,0.25507
validation/accuracy,80.06
validation/loss,0.58171


test/accuracy,▁
test/loss,▁
test/accuracy,81.6
test/loss,0.54834


##Semi

In [ ]:

if (task == "semi"):

  torch.manual_seed(seed)
  np.random.seed(seed)

  # Training the labeller
  lab_train_tot_accs, lab_valid_tot_accs = [], []
  lab_train_tot_losses, lab_valid_tot_losses = [], []

  Labeller = ModelClass(optimizer=optimizer,lr=learn_rate,weight_decay=weight_decay)
  Labeller = Labeller.to(device=device)
  
  for ep in range(epoch):
            
    # logging.info(f"==========Semi-supervised Learning Labeller Epoch Number: {ep+1}/{epoch}==========")
    print(f"==========Semi-supervised Learning Labeller Epoch Number: {ep+1}/{epoch}==========")
    train_accs, valid_accs = [], []
    train_losses, valid_losses = [], []

    if ep == epoch/2:
      learn_rate = learn_rate/10
      print(f"Update Learning Rate: {learn_rate}")
      Labeller.update_lr(learn_rate)

    if ep == 3*epoch/4:
      learn_rate = learn_rate/10
      print(f"Update Learning Rate: {learn_rate}")
      Labeller.update_lr(learn_rate)


    for idx, batch in enumerate(labelledloader):
      data, target = batch
      data = data.to(device=device)
      labels = F.one_hot(target, num_classes = 10).float().to(device=device)
      batch_len = data.shape[0]
      aug_num = []

      if augment_file != None:

        for i in range(len(augment_strength)):
          aug_num.append(augment_strength[i]*batch_len)

        if len(aug_num) != 1:
          aug_num = torch.tensor(aug_num)
          aug_ind = torch.cumsum(aug_num,0).int()
        else:
          aug_ind = aug_num

        for i in range(len(Aug)):
          if i == 0:
            temp_Aug, temp_label = Aug[i](data[0:aug_ind[i]],labels[0:aug_ind[i]])
            Aug_data = temp_Aug
            Aug_labels = temp_label
          else:
            temp_Aug, temp_label = Aug[i](data[aug_ind[i-1]:aug_ind[i]],labels[aug_ind[i-1]:aug_ind[i]])
            Aug_data = torch.cat((Aug_data, temp_Aug), 0)
            Aug_labels = torch.cat((Aug_labels, temp_label), 0)

      else:
        Aug_data = torch.cat((data,data),0)
        Aug_labels = torch.cat((labels,labels),0)


      acc, loss = Labeller.train_sup_up(Aug_data,Aug_labels)
      train_accs.append(acc)
      train_losses.append(loss)
      
      lab_train_tot_accs.append(sum(train_accs)/len(train_accs))
      lab_train_tot_losses.append(sum(train_losses)/len(train_losses))
          
    # logging.info(f"==========Training Accuracy: {lab_train_tot_accs[-1]:.3f} , Training Loss: {lab_train_tot_losses[-1]:.3f}==========")    
    print(f"==========Training Accuracy: {lab_train_tot_accs[-1]:.3f} , Training Loss: {lab_train_tot_losses[-1]:.3f}==========")

    for idx, batch in enumerate(validloader):
      data, target = batch
      data = data.to(device=device)
      labels = F.one_hot(target, num_classes = 10).float().to(device=device)
      acc, loss = Labeller.evaluation(data,labels)
      valid_accs.append(acc)
      valid_losses.append(loss)

    lab_valid_tot_accs.append(sum(valid_accs)/len(valid_accs))
    lab_valid_tot_losses.append(sum(valid_losses)/len(valid_losses))
      
    # logging.info(f"==========Validation Accuracy: {lab_valid_tot_accs[-1]:.3f} , Validation Loss: {lab_valid_tot_losses[-1]:.3f}==========")    
    print(f"==========Validation Accuracy: {lab_valid_tot_accs[-1]:.3f} , Validation Loss: {lab_valid_tot_losses[-1]:.3f}==========")
  





          
  # Accuracy and loss when predicting the unlabelled data
  lab_accs = []
  lab_losses = []
  
  for idx, batch in enumerate(unlabelledloader):
    data, target = batch
    data = data.to(device=device)
    labels = F.one_hot(target, num_classes = 10).float().to(device=device)
    acc, loss = Labeller.evaluation(data,labels)
    lab_accs.append(acc)
    lab_losses.append(loss)
          
  lab_tot_accs = (sum(lab_accs)/len(lab_accs))
  lab_tot_losses = (sum(lab_losses)/len(lab_losses))
              
  # logging.info(f"==========Labelled Accuracy: {lab_tot_accs:.3f} , Labelled Loss: {lab_tot_losses:.3f}==========")           
  print(f"==========Labelled Accuracy: {lab_tot_accs:.3f} , Labelled Loss: {lab_tot_losses:.3f}==========")           

  
  # Train the final model with labelled data and unlabelled data where the target is predicted by the the labeller
  train_tot_accs, valid_tot_accs = [], []
  train_tot_losses, valid_tot_losses = [], []

  Model = ModelClass(optimizer=optimizer,lr=learn_rate,weight_decay=weight_decay)
  Model = Model.to(device=device)
  
  for ep in range(epoch):
      
    # logging.info(f"==========Semi-supervised Learning Model Epoch Number: {ep+1}/{epoch}==========")
    print(f"==========Semi-supervised Learning Model Epoch Number: {ep+1}/{epoch}==========")
    train_accs, valid_accs = [], []
    train_losses, valid_losses = [], []
    
    if ep == epoch/2:
      learn_rate = learn_rate/10
      print(f"Update Learning Rate: {learn_rate}")
      Model.update_lr(learn_rate)

    if ep == 3*epoch/4:
      learn_rate = learn_rate/10
      print(f"Update Learning Rate: {learn_rate}")
      Model.update_lr(learn_rate)

    for idx, batch in enumerate(labelledloader):
      data, target = batch
      data = data.to(device=device)
      labels = F.one_hot(target, num_classes = 10).float().to(device=device)
      batch_len = data.shape[0]
      aug_num = []

      if augment_file != None:

        for i in range(len(augment_strength)):
          aug_num.append(augment_strength[i]*batch_len)

        if len(aug_num) != 1:
          aug_num = torch.tensor(aug_num)
          aug_ind = torch.cumsum(aug_num,0).int()
        else:
          aug_ind = aug_num

        for i in range(len(Aug)):
          if i == 0:
            temp_Aug, temp_label = Aug[i](data[0:aug_ind[i]],labels[0:aug_ind[i]])
            Aug_data = temp_Aug
            Aug_labels = temp_label
          else:
            temp_Aug, temp_label = Aug[i](data[aug_ind[i-1]:aug_ind[i]],labels[aug_ind[i-1]:aug_ind[i]])
            Aug_data = torch.cat((Aug_data, temp_Aug), 0)
            Aug_labels = torch.cat((Aug_labels, temp_label), 0)

      else:
        Aug_data = torch.cat((data,data),0)
        Aug_labels = torch.cat((labels,labels),0)


      acc, loss = Model.train_sup_up(Aug_data,Aug_labels)
      train_accs.append(acc)
      train_losses.append(loss)



    for idx, batch in enumerate(unlabelledloader):
      data, _ = batch
      data = data.to(device=device)
      target = torch.argmax(Labeller.forward(data), dim=1)
      labels = F.one_hot(target, num_classes = 10).float().to(device=device)

      batch_len = data.shape[0]
      aug_num = []

      if augment_file != None:

        for i in range(len(augment_strength)):
          aug_num.append(augment_strength[i]*batch_len)

        if len(aug_num) != 1:
          aug_num = torch.tensor(aug_num)
          aug_ind = torch.cumsum(aug_num,0).int()
        else:
          aug_ind = aug_num

        for i in range(len(Aug)):
          if i == 0:
            temp_Aug, temp_label = Aug[i](data[0:aug_ind[i]],labels[0:aug_ind[i]])
            Aug_data = temp_Aug
            Aug_labels = temp_label
          else:
            temp_Aug, temp_label = Aug[i](data[aug_ind[i-1]:aug_ind[i]],labels[aug_ind[i-1]:aug_ind[i]])
            Aug_data = torch.cat((Aug_data, temp_Aug), 0)
            Aug_labels = torch.cat((Aug_labels, temp_label), 0)

      else:
        Aug_data = torch.cat((data,data),0)
        Aug_labels = torch.cat((labels,labels),0)


      acc, loss = Model.train_sup_up(Aug_data,Aug_labels)
      train_accs.append(acc)
      train_losses.append(loss)
      
    train_tot_accs.append(sum(train_accs)/len(train_accs))
    train_tot_losses.append(sum(train_losses)/len(train_losses))
          
    # logging.info(f"==========Training Accuracy: {train_tot_accs[-1]:.3f} , Training Loss: {train_tot_losses[-1]:.3f}==========")    
    print(f"==========Training Accuracy: {train_tot_accs[-1]:.3f} , Training Loss: {train_tot_losses[-1]:.3f}==========")   




    for idx, batch in enumerate(validloader):
      data, target = batch
      data = data.to(device=device)
      labels = F.one_hot(target, num_classes = 10).float().to(device=device)
      acc, loss = Model.evaluation(data,labels)
      valid_accs.append(acc)
      valid_losses.append(loss)

    valid_tot_accs.append(sum(valid_accs)/len(valid_accs))
    valid_tot_losses.append(sum(valid_losses)/len(valid_losses))
    
    # logging.info(f"==========Validation Accuracy: {valid_tot_accs[-1]:.3f} , Validation Loss: {valid_tot_losses[-1]:.3f}==========")    
    print(f"==========Validation Accuracy: {valid_tot_accs[-1]:.3f} , Validation Loss: {valid_tot_losses[-1]:.3f}==========")
  
  

  print(f"==========Max Training Accuracy: {torch.max(torch.tensor(train_tot_accs)):.3f} , Min Training Loss: {torch.min(torch.tensor(train_tot_losses)):.3f}==========")    
  print(f"==========Max Validation Accuracy: {torch.max(torch.tensor(valid_tot_accs)):.3f} , Min Validation Loss: {torch.min(torch.tensor(valid_tot_losses)):.3f}==========")

==========Semi-supervised Learning Labeller Epoch Number: 1/160==========
==========Training Accuracy: 0.264 , Training Loss: 1.969==========
==========Validation Accuracy: 0.321 , Validation Loss: 1.834==========
==========Semi-supervised Learning Labeller Epoch Number: 2/160==========
==========Training Accuracy: 0.360 , Training Loss: 1.715==========
==========Validation Accuracy: 0.388 , Validation Loss: 1.664==========
==========Semi-supervised Learning Labeller Epoch Number: 3/160==========
==========Training Accuracy: 0.419 , Training Loss: 1.549==========
==========Validation Accuracy: 0.393 , Validation Loss: 1.663==========
==========Semi-supervised Learning Labeller Epoch Number: 4/160==========
==========Training Accuracy: 0.474 , Training Loss: 1.428==========
==========Validation Accuracy: 0.466 , Validation Loss: 1.455==========
==========Semi-supervised Learning Labeller Epoch Number: 5/160==========
==========Training Accuracy: 0.519 , Training Loss: 1.313==========
==

## Plots

In [ ]:
test_accs = []
test_losses = []

for idx, batch in enumerate(testloader):
    data, target = batch
    data = data.to(device=device)
    labels = F.one_hot(target, num_classes = 10).float().to(device=device)
    acc, loss = Model.evaluation(data,labels)
    test_accs.append(acc)
    test_losses.append(loss)
        
test_tot_accs = (sum(test_accs)/len(test_accs))
test_tot_losses = (sum(test_losses)/len(test_losses))
            
# logging.info(f"==========Test Accuracy: {test_tot_accs:.3f} , Test Loss: {test_tot_losses:.3f}==========") 
print(f"==========Test Accuracy: {test_tot_accs:.3f} , Test Loss: {test_tot_losses:.3f}==========")  

In [ ]:
import matplotlib.pyplot as plt
plt.plot(torch.stack(train_tot_accs).cpu().detach().numpy(), label="Train")
plt.plot(torch.stack(valid_tot_accs).cpu().detach().numpy(), label="Valid")
plt.title('Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:

plt.plot(torch.stack(train_tot_losses).cpu().detach().numpy(), label="Train")
plt.plot(torch.stack(valid_tot_losses).cpu().detach().numpy(), label="Valid")
plt.title('Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
if not os.path.exists(f"./src/Evaluation/Logs/{config_name}"):
    os.makedirs(f"./src/Evaluation/Logs/{config_name}")
with open(f"./src/Evaluation/Logs/{config_name}/{config_name}_train_acc.txt", "w") as f:
        f.write("\n".join(str(item.cpu().detach().numpy()) for item in train_tot_accs))
with open(f"./src/Evaluation/Logs/{config_name}/{config_name}_train_loss.txt", "w") as f:
        f.write("\n".join(str(item.cpu().detach().numpy()) for item in train_tot_losses))
with open(f"./src/Evaluation/Logs/{config_name}/{config_name}_valid_acc.txt", "w") as f:
        f.write("\n".join(str(item.cpu().detach().numpy()) for item in valid_tot_accs))
with open(f"./src/Evaluation/Logs/{config_name}/{config_name}_valid_loss.txt", "w") as f:
        f.write("\n".join(str(item.cpu().detach().numpy()) for item in valid_tot_losses))
with open(f"./src/Evaluation/Logs/{config_name}/{config_name}_test_acc.txt", "w") as f:
        f.write("\n".join(str(item.cpu().detach().numpy()) for item in [test_tot_accs]))
with open(f"./src/Evaluation/Logs/{config_name}/{config_name}_test_loss.txt", "w") as f:
        f.write("\n".join(str(item.cpu().detach().numpy()) for item in [test_tot_losses]))

In [ ]:
# pickle_path = f"./Model/{args.config_file}.pickle"

if not os.path.exists(f"./src/Model/{model_file}"):
    os.makedirs(f"./src/Model/{model_file}")
pickle_path = f"./src/Model/{model_file}/{config_name}.pickle"
logging.info("Saving model to pickle file")
with open(pickle_path, "wb") as f:
  pickle.dump(Model, f, pickle.HIGHEST_PROTOCOL)



